# SageMaker Demo
This notebook is intended to be used with a [SageMaker notebook instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html) launched using the following [CloudFormation](https://docs.aws.amazon.com/cloudformation/) template:

- [sagemaker-notebook-cloudformation.yml](https://github.com/managedkaos/jupyter-environment-details/blob/main/sagemaker-notebook-cloudformation.yml)

Together the CloudFormation template and this notebook demonstrate:

- Attaching an IAM role to a SageMaker instance with policies that allow the instance to use other AWS services
- Using the [Boto3 Python library](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to create clients for accessing AWS services
- Using boto3 clients to read from [Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) and write to an [S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html)



## THERE IS ONE PLACE WHERE THE NOTEBOOK NEEDS TO BE UPDATED TO WORK CORRECTLY

_In cell #3, update the value for `NotebookInstanceName` with the value you used in CloudFormation._

## Install the Boto3 library and initialize clients for S3 and SSM

In [ ]:
%pip install --quiet boto3

In [ ]:
import boto3
s3_client = boto3.client('s3')
ssm_client = boto3.client('ssm')

## Update the code to use the parameter value for `NotebookInstanceName`

In [ ]:
# Replace with your value for NotebookInstanceName
NotebookInstanceName = "sagemaker-demo-202403112200"

## Create helper functions
- read_from_parameter_store(name)
- write_to_s3(bucket, key, content)

In [ ]:
# Read value from Parameter Store
def read_from_parameter_store(name):
    response = ssm_client.get_parameter(Name=name, WithDecryption=True)
    return response['Parameter']['Value']

# Write data to the S3 bucket
def write_to_s3(bucket, key, body):
    s3_client.put_object(Bucket=bucket, Key=key, Body=body)
    print(f"Successfully wrote data to s3://{bucket}/{key}")


## Read from SSM ParameterStore 

In [ ]:
# Constructed Parameter Names
s3_bucket_name_parameter = f"/{NotebookInstanceName}/S3Bucket"

# Get the S3 bucket name from Parameter Store
bucket_name = read_from_parameter_store(s3_bucket_name_parameter)
print(f"S3 Bucket Name from ParameterStore: {bucket_name}")

## Create `index.html` in the S3 bucket

In [ ]:
# Define the object key and file content
write_to_s3(bucket_name, "index.html", f"<h1>Welcome to data store for {NotebookInstanceName}</h1>")

## Write another file to the bucket

In [ ]:
write_to_s3(bucket_name, "data.txt", "sample data goes here")